### Simulated Annealing in Julia for the Number Partitioning Problem

#### Problem set and initialising initial state

In [58]:
# Problem set to partition
numbers = [4, 2, 7, 1]

# Initialising a random state of spins = {-1, 1}
state_0 = [Int(rand(Int64(0):Int64(1))) for i = 1:size(numbers,1)]
for i in 1:size(numbers,1)
    if state_0[i] == 0
       state_0[i] -= 1
    end
end

# Number of iterations of SA (terminating control)
iterations = 1000;

#### Neighbour state function

In [59]:
function neighbour(s::Vector{Int64}) # Moves the current state to a neighbouring state   
    s_proposal = s
    
    # Keeps the first spin value constant to remove double degeneracy of solutions 
    # From the paper “Ising formulations of many NP problems” [Lucas]
    for i in 2:size(s, 1)
        random = Int64(rand(Int64(0):Int64(1)))
        if s[i] == -1
            s_proposal[i] += random
            if random == 1
                s_proposal[i] += 1       
            end                
        else
            s_proposal[i] -= random
            if random == 1
                s_proposal[i] -= 1
            end                    
        end
    end
    return s_proposal
end

neighbour (generic function with 1 method)

#### Cost function 

In [62]:
function cost(numbers, state)
    cost_root = 0
    for i in 1:size(state, 1)
        cost_root += numbers[i]*state[i]
    end
    cost = cost_root^2
    return cost
end

cost (generic function with 1 method)

#### Cooling schedules

In [63]:
function log_temp(i)
    return 1 / log(i)
end

function constant_temp(i)
    return 1
end


constant_temp (generic function with 2 methods)

#### Simulated annealing function

In [67]:
function simulated_annealing(state_0::Vector{Int64},
                             numbers::Vector{Int64},
                             iterations::Int64,
                             cost::Function,
                             neighbour::Function,
                             temperature::Function,
                             keep_best::Bool,
                             trace::Bool)
    state = state_0
    best_state = state_0
    x = copy(numbers)
    flag = 0
    temp = 10
    
    for i = 1:iterations
        
        temp = temperature(i)
        state_temp_1 = copy(state)
        state_temp_2 = copy(state)
        y = cost(x, state_temp_1)
        
        if  y == 0
            best_state = state
            flag = 1
            if keep_best
                println()
                println("problem : $x")
                println("solution: $best_state")
                println("cost    : $y")
            break
            end
        end
        
        if trace println("$i: state = $state") end
        if trace println("$i: y = $y") end
        
        state_n = neighbour(state_temp_2)
        y_n = cost(x, state_n)
        
        if  y_n == 0
            best_state = state_n
            flag = 1
            if keep_best
                println()
                println("problem : $x")
                println("solution: $best_state")
                println("cost    : $y_n")
            break
            end
        end
        
        if trace println("$i: state_n = $state_n") end
        if trace println("$i: y_n = $y_n") end
        
        if y_n <= y
            state = state_n
            if trace println("accept state") end
        else
            probability = exp( - ((y_n - y) / temp))
            if trace println("$i: probability = $probability") end
            if rand() <= probability
                state = state_n
                if trace println("accept state") end
            else
                state = state
                if trace println("reject state") end
            end
        end
    
        if cost(x, state) < cost(x, best_state)
            best_state = state
        end
    
        if trace println() end
    end
  
    if flag == 0
        println()
        if keep_best
            println("problem : $x")
            println("solution: $best_state")
        else
            println("problem : $x")
            println("solution: $state")
        end
    end
end

simulated_annealing (generic function with 1 method)

#### Solution to the number partitioning problem

In [68]:
solution = simulated_annealing(state_0,     # initial state of spins = {-1, 1} (length of problem set)
                               numbers,     # problem set to be partitioned
                               iterations,  # number of iterations of SA (only termination condition)
                               cost,        # cost function derived from the Ising 
                               neighbour,   # function that moves the state to a neighbouring state
                               log_temp,    # cooling schedule
                               true,        # choice to keep the best state  
                               true)        # choice to trace the SA steps

1: state = [-1, 1, -1, -1]
1: y = 100
1: state_n = [-1, -1, -1, 1]
1: y_n = 144
1: probability = 1.0
accept state

2: state = [-1, -1, -1, 1]
2: y = 144
2: state_n = [-1, -1, -1, 1]
2: y_n = 144
accept state

3: state = [-1, -1, -1, 1]
3: y = 144
3: state_n = [-1, -1, 1, 1]
3: y_n = 4
accept state

4: state = [-1, -1, 1, 1]
4: y = 4
4: state_n = [-1, 1, 1, -1]
4: y_n = 16
4: probability = 5.960464477539072e-8
reject state

5: state = [-1, -1, 1, 1]
5: y = 4
5: state_n = [-1, 1, 1, 1]
5: y_n = 36
5: probability = 4.2949672960000126e-23
reject state

6: state = [-1, -1, 1, 1]
6: y = 4
6: state_n = [-1, -1, -1, 1]
6: y_n = 144
6: probability = 1.145051305641961e-109
reject state

7: state = [-1, -1, 1, 1]
7: y = 4
7: state_n = [-1, 1, -1, -1]
7: y_n = 100
7: probability = 7.423148669824725e-82
reject state

8: state = [-1, -1, 1, 1]
8: y = 4
8: state_n = [-1, 1, -1, 1]
8: y_n = 64
8: probability = 6.525304467998561e-55
reject state

9: state = [-1, -1, 1, 1]
9: y = 4
9: state_n = [-1, 1, 